# General Information

This is the first exercise in the KI course. You will have to implement a few algorithms from the lecture. The assignment will be graded, but it should also serve for you to understand the concepts more deeply and experiment and play around with the ideas. 

You can add code and cells at any point in the notebook; the only important thing for the grading to work is that you don't change the function names. The signature of the functions (the meaning and order of function arguments and return values is given, and should not be changed). Please do not use special ipython functionality, such as [ipython magics](http://ipython.org/ipython-doc/dev/interactive/tutorial.html).

Practical help with the assignments will be available in moodle and during the tutorial sessions, which will be anounced on moodle and on the KI website.

Some of the exercises will not get a visible grade on the course website, and will only be graded after the assignment deadline. Also, the grades might change slightly if we realize that our evaluation was not optimal.

# The GSAT algorithm

In this exercise, you will implement an algorithm to find an approximate solution for the 3-CNF problem, that is, the problem of finding a satisfying assignment for a logical sentence of the form 
$$
\left(P \vee Q \vee \neg S\right) \wedge \left(R \vee T \vee \neg P\right) \wedge \left(R \vee T \vee \neg S\right).
$$

The GSAT algorithm is an instance of a class of algorithms called [*Local Search Algorithms*](http://en.wikipedia.org/wiki/WalkSAT), which explore the state space of boolean assignments to variables by considering a locally optimal, or *greedy* change to that assignment.

3-CNF formulas are in CNF form, and each clause contains exactly 3 literals. The GSAT algorithm starts with a random truth assignment to the symbols, and then tries to find a satisfying assignment by changing one truth value at a time. The symbol to be changed is selected such that the maximum number of clauses is satisfied after each step. If there are multiple variables that satisfy the same number of clauses, the decision between them is arbitrary.

Since the trajectory of the state (the truth assignments to each variable in each step of the algorithm) depends heavily on the initial state, the algorithm is restarted several times with different initial states.

The algorithm performance is influenced by two important parameters: $C$, the number of clauses, and $N$, the number of proposition symbols.

## Representing 3-CNF problems

For this exercise, 3-CNFs are represented as a set of clauses, where each clause consists of two sets: The first one contains the indices of the variables in the clause that are nonnegated; the second one the indices for the variables that appear negated. The state is simply a list of boolean values, of length N. 

For the example given above, that would be:

In [158]:
state = [False, False, True, True, False]

# variables in the order (P, Q, R, S, T)
problem = [
           ({0, 1}, {3, }),
           ({2, 4}, {0, }),
           ({2, 4}, {3, })
           ]
           

## Complexity

Which complexity class does the problem of satifisbility of 3-CNF formulas belong to? Select the most concise answer. Return a string from your function.

In [159]:
def three_cnf_complexity():
    return 'NP hard'

How many evaluations have to be made in every step of the algorithm (assuming that the satisfiability of a clause can be checked in 1 step)? Return a sympy function in the variables $N$ and $C$.

In [160]:
from sympy import var
var('N C')
def gsat_step_complexity(N, C):
    return N * C

Is this algorithm complete? Return `True` or `False` (as `bool` values).

In [161]:
def gsat_complete():
    return False

## Generating Problems

Generate random instances of 3-CNF problems, given the number of clauses `n_clauses` and the number of variables `n_vars`. Note that the representation of the positive and negative literals for each clause as sets does not allow clauses like $P \wedge P \wedge Q$. Your random problems should always have 3 different literals in the set representation.

In [235]:
%autosave 0
import random as rd
import numpy as np
def generate_random_problem(n_vars, n_clauses):
    num_firstClass = [rd.randint(0,3) for i in xrange(n_clauses)]
    initialState = [rd.sample(range(0, n_vars), 3) for i in xrange(n_clauses)]
    initialState = np.ravel(initialState)

    problem  = []
    for i in xrange(n_clauses):
        set1 = set(initialState[3 * i + j] for j in xrange(num_firstClass[i]))
        set2 = set(initialState[3 * i + num_firstClass[i] + j] for j in xrange(3 - num_firstClass[i]))
        problem.append((set1, set2))
            
    return problem

Autosave disabled


Can you think of a simple way to simplify the problem in cases where clauses are tautological?
Write a function that simplifies the problem accordingly.

In [236]:
def simplify_three_cnf(problem):
    simplified_problem = []

    for clauseInPro in problem:
        if not(0 < len(clauseInPro[0]) < 3):
            simplified_problem.append(clauseInPro)
            continue
        if not(clauseInPro[0] & clauseInPro[1]):
            simplified_problem.append(clauseInPro)

    return simplified_problem

## Implementing GSAT

Write a function that generates the initial state for a 3-CNF SAT problem. It should be truly random, so that calling it multiple times gives different results.

In [224]:
def get_initial_state(n_vars, n_clauses):
    initial_state = [bool(rd.randint(0, 1)) for i in xrange(n_vars)]
    return initial_state

Now, write a function that evaluates the truth value of a single clause, and returns whether it is satisfied:

In [225]:
def eval_clause(state, clause):
    for elementSet1 in clause[0]:
        if state[elementSet1]:
            return True
    for elementSet2 in clause[1]:
        if not(state[elementSet2]):
            return True
    return False

Building on this, add a function that evaluates the truth value of a whole 3-CNF formula `problem` given the `state`:

In [226]:
def eval_three_cnf(problem, state):
    for clauseInPro in problem:
        if not(eval_clause(state, clauseInPro)):
            return False
    return True

### Stopping Condition

Write a function that checks if a solution, i.e. a state that satisfies all clauses, has been found. The function should return the Boolean value `True` if the algorithm is done, and `False` otherwise.

In [227]:
def am_i_done(problem, state):
    if eval_three_cnf(problem, state):
        return True
    return False

## Putting together the algorithm

Write a function that runs one chain of GSAT for a given maximum number of iterations `max_iter`. It should return the best encountered state and whether the algorithm succeeded in finding a satisfying assignment or not, and it should return as early as possible.

In [232]:
import copy as cp
def run_gsat_chain(problem, state, max_iter):

    if am_i_done(problem, state):
        return state, True
    
    numTrueClausesFinal = 0
    
    for i in xrange(max_iter):
        if i > len(state) - 1:
            break

        realState = cp.copy(state)
        realState[i] = not(realState[i])

        if am_i_done(problem, realState):
            return realState, True

        numTrueClauses = 0
        for clausesInPro in problem:
            if eval_clause(realState, clausesInPro):
                numTrueClauses = numTrueClauses + 1

        if numTrueClausesFinal <= numTrueClauses:
            stateFinal = realState
            numTrueClausesFinal = numTrueClauses

    return stateFinal, False

Now, write a function that generates an initial state in `n_vars` variables for the multiple chains (at most `max_n_chains` of them), runs each of the chains, and returns `success` (as a Boolean variable) and a satisfying assignment if there was one, or else the best assignment that was found.

In [234]:
def run_gsat(problem, max_iter, n_vars, max_n_chains):
    numTrueClausesFinal = 0
    numClauses = len(problem)
    for i in xrange(max_n_chains):
        state, success = run_gsat_chain(problem, get_initial_state(n_vars, numClauses), max_iter)
        if success:
            return True, state

        numTrueClauses = 0
        
        for clauseInPro in problem:
            if eval_clause(state, clauseInPro):
                numTrueClauses = numTrueClauses + 1

        if numTrueClausesFinal <= numTrueClauses:
            numTrueClausesFinal = numTrueClauses
            satisfying_assignment = state

    return False, satisfying_assignment

(True, [False, True, True, True, False, False, True, False, False, False])


## Experiment

Experiment! Generate random problems of different sizes by varying $C$ and $N$ for your assignment function and use the timing functions of python to check the runtimes of the algorithm for different problems, and determine what feasible values for `max_iter` and `max_n_chains` could be. Make a plot of typical runtimes and their statistics (`np.mean` and `np.median` can be useful here) versus the algorithm parameters.

Timing a function works like so:

```
def foo():
    pass

import timeit
timeit.timeit(foo)    
```


In [206]:
def foo():
    pass

import timeit
timeit.timeit(foo)

0.16141796112060547